# INFO

- ĐỒ ÁN CUỐI KÌ MÔN `LẬP TRÌNH CHO KHOA HỌC DỮ LIỆU`
- Thành viên:
    + 18120517 Nguyễn Công Bình Phương
    + 18120514 Phan Thiên Phúc

# OUR ANALYSIS

In [1]:
#import additional packages to env
!conda install -c conda-forge seaborn -y

Solving environment: done

## Package Plan ##

  environment location: /Users/nathan/opt/anaconda3

  added / updated specs:
    - seaborn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.10.3               |   py38h50d1736_0         3.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be INSTALLED:

  python_abi         conda-forge/osx-64::python_abi-3.8-2_cp38

The following packages will be SUPERSEDED by a higher-priority channel:

  conda              pkgs/main::conda-4.10.3-py38hecd8cb5_0 --> conda-forge::conda-4.10.3-py38h50d1736_0



conda-4.10.3         | 3.1 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [2]:
#import library
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

## ABOUT DATA

nguồn: https://www.kaggle.com/russellyates88/suicide-rates-overview-1985-to-2016

- Content: This compiled dataset pulled from four other datasets linked by time and place, and was built to find signals correlated to increased suicide rates among different cohorts globally, across the socio-economic spectrum.

- References: 
    United Nations Development Program. (2018). Human development index (HDI). Retrieved from http://hdr.undp.org/en/indicators/137506

    World Bank. (2018). World development indicators: GDP (current US$) by country:1985 to 2016. Retrieved from http://databank.worldbank.org/data/source/world-development-indicators#

    [Szamil]. (2017). Suicide in the Twenty-First Century [dataset]. Retrieved from https://www.kaggle.com/szamil/suicide-in-the-twenty-first-century/notebook

    World Health Organization. (2018). Suicide prevention. Retrieved from http://www.who.int/mental_health/suicide-prevention/en/

- Inspiration: Suicide Prevention.


***Data dictionary***:
- Country: Column representing countries (string)
- Year: Column representing years (integer)
- Sex: Column representing genres (string)
- Age: Column representing ages (string)
- Suicides_no: Column representing the number of suicides (integer)
- Population: Column representing the population (integer)
- Suicides / 100k pop: Column that represents the number of suicides per 100 thousand inhabitants (float)
- Country-year: Column that represents (string)
- HDI for year: Column that represents the HDI in each year (float)
- Gdp_for_year($): Column representing GDP in each year (string)
- Gdp_per_capita($): Column representing GDP per capita (float)
- Generation: Column that represents the generation (string)

## DATA ANALYSIS

### Đọc file dữ liệu

In [3]:
df = pd.read_csv('master.csv')
df.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


### Dữ liệu gồm có bao nhiêu dòng và bao nhiêu cột?

In [4]:
df.shape

(27820, 12)

### Kiểu dữ liệu của từng cột

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27820 entries, 0 to 27819
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   country             27820 non-null  object 
 1   year                27820 non-null  int64  
 2   sex                 27820 non-null  object 
 3   age                 27820 non-null  object 
 4   suicides_no         27820 non-null  int64  
 5   population          27820 non-null  int64  
 6   suicides/100k pop   27820 non-null  float64
 7   country-year        27820 non-null  object 
 8   HDI for year        8364 non-null   float64
 9    gdp_for_year ($)   27820 non-null  object 
 10  gdp_per_capita ($)  27820 non-null  int64  
 11  generation          27820 non-null  object 
dtypes: float64(2), int64(4), object(6)
memory usage: 2.5+ MB


**Nhận xét:** Các cột có kiểu dữ liệu phù hợp với dữ liệu cung cấp. Tuy nhiên, ta thấy có cột `country-year` khá thừa khi đã có cột `country` và `year` độc lập. Ta sẽ loại bỏ cột `country-year` ở trong bước tiền xử lý dữ liệu.

### Dữ liệu có các dòng bị lặp hay rỗng không?

In [6]:
df.duplicated().any()

False

=> Không có dữ liệu lặp

In [7]:
df.isnull().values.any()

True

=> Trong tập dữ liệu có các điểm dữ liệu thiếu => Ta sẽ cùng tìm hiểu kĩ hơn

In [15]:
def count_missing_values(s):
    return s.isna().sum()

def calculate_missing_percentage(s):
    return s.isna().sum() / df.shape[0] * 100

df.agg([count_missing_values, calculate_missing_percentage])

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
count_missing_values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19456.000000,0.0,0.0,0.0
calculate_missing_percentage,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,69.935298,0.0,0.0,0.0


**Nhận xét:** Ta thấy rằng trong tập dữ liệu chỉ có duy nhất cột `HDI for year` có giá trị thiếu và  bị thiếu dữ liệu rất nhiều (~ 70%).
`HDI for year` ở đây là chỉ số `Human Development Index` của đất nước tại thời điểm được xét `year`. Tìm thiểu thêm về HDI [tại đây](https://vi.wikipedia.org/wiki/Ch%E1%BB%89_s%E1%BB%91_ph%C3%A1t_tri%E1%BB%83n_con_ng%C6%B0%E1%BB%9Di)

Vậy nên ta sẽ loại bỏ cột này ở bước tiền xử lý dữ liệu

## INSIGHT

Trước khi bước đến bước tiền xử lý dữ liệu, dựa vào những khám phá và phân tích dữ liệu cơ bản ở các bước trên. Ta sẽ chọn insight sau

***Sự ảnh hưởng của việc tự tử ở người dân đối với nền kinh tế các nước***


Dự đoán của bản thân trước khi phân tích:
```
Có thể việc tự tử ở người dân sẽ chịu ảnh hưởng và gây ảnh hưởng đến nền kinh tế của một nước.
```